In [1]:
import os
#from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
#from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.preprocessing.image import ImageDataGenerator, load_img
#from keras.applications.resnet50 import ResNet50
#from keras.preprocessing import image
#from keras.models import Sequential
#from keras.models import Model
#from keras.callbacks import ModelCheckpoint, EarlyStopping
#import matplotlib.pyplot as plt
#import numpy as np
import joblib


from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import os
import numpy as np
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Image preprocessing
def preprocess(train_data_dir, valid_data_dir, test_data_dir):
    img_height, img_width = (224, 224)
    batch_size = 32

    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       validation_split=0.4)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    valid_generator = train_datagen.flow_from_directory(
        valid_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')

    test_generator = train_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical',
        subset='validation')

    return train_generator, test_generator, valid_generator

def extract_features(base_model, generator):
    features = []
    labels = []
    for i in range(len(generator)):
        x, y = generator[i]
        feature_batch = base_model.predict(x)
        label_batch = np.argmax(y, axis=1)
        features.append(feature_batch)
        labels.append(label_batch)
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

def cnn_model():
    base_model = ResNet101(include_top=False, weights='imagenet')
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)  # Assuming 2 classes for binary classification
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

def train_cnn_model(train_generator, valid_generator):
    model = cnn_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(
        train_generator,
        epochs=10,
        validation_data=valid_generator,
        verbose=1
    )
    return model

def train_xgb_model(train_features, train_labels, valid_features, valid_labels):
    dtrain = xgb.DMatrix(train_features, label=train_labels)
    dvalid = xgb.DMatrix(valid_features, label=valid_labels)

    params = {
        'objective': 'multi:softmax',
        'num_class': len(np.unique(train_labels)),
        'eval_metric': 'mlogloss'
    }

    evals = [(dtrain, 'train'), (dvalid, 'eval')]
    num_round = 100
    xgb_model = xgb.train(params, dtrain, num_round, evals, early_stopping_rounds=10, verbose_eval=10)
    return xgb_model

if __name__ == '__main__':
    train_data_dir = "/workspace/data/data_dir/model-data/train"
    test_data_dir = "/workspace/data/data_dir/model-data/test"
    valid_data_dir = "/workspace/data/data_dir/model-data/val"

    train_generator, test_generator, valid_generator = preprocess(train_data_dir, test_data_dir, valid_data_dir)

    cnn_trained_model = train_cnn_model(train_generator, valid_generator)

    # Extract features
    train_features, train_labels = extract_features(cnn_trained_model, train_generator)
    valid_features, valid_labels = extract_features(cnn_trained_model, valid_generator)

    # Train XGBoost model
    xgb_model = train_xgb_model(train_features, train_labels, valid_features, valid_labels)

    # Evaluate XGBoost model
    test_features, test_labels = extract_features(cnn_trained_model, test_generator)
    dtest = xgb.DMatrix(test_features, label=test_labels)
    preds = xgb_model.predict(dtest)
    accuracy = np.sum(preds == test_labels) / len(test_labels)
    print("XGBoost Accuracy:", accuracy)


Found 1051 images belonging to 3 classes.
Found 200 images belonging to 3 classes.
Found 99 images belonging to 3 classes.


2024-05-03 08:20:27.908646: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 08:20:28.787649: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2024-05-03 08:20:28.787841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 918 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:90:00.0, compute capability: 8.0


Epoch 1/10


2024-05-03 08:20:37.651560: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8500
2024-05-03 08:20:39.217125: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-05-03 08:20:39.221171: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Graph execution error:

Detected at node 'binary_crossentropy/logistic_loss/mul' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyn

In [3]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

def preprocess(train_data_dir, valid_data_dir, test_data_dir):
    img_height, img_width = (227, 227)  # Image dimensions assumed for AlexNet
    batch_size = 32

    train_datagen = ImageDataGenerator(rescale=1./255,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       validation_split=0.4)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    valid_generator = train_datagen.flow_from_directory(
        valid_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')

    test_generator = train_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical',
        subset='validation')

    return train_generator, test_generator, valid_generator

def alexnet_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(96, (11, 11), strides=(4, 4), activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(256, (5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(384, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(384, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = BatchNormalization()(x)

    # Fully connected layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output layer
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=predictions)

    return model

def train_cnn_model(train_generator, valid_generator, num_classes):
    base_model = alexnet_model((227, 227, 3), num_classes)
    base_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint_callback = ModelCheckpoint('/workspace/data/data_dir/saved/best/best_model.h5',
                                          monitor='val_loss', save_best_only=True, verbose=1)
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    
    log_dir = "/workspace/data/data_dir/logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    base_model.fit(
        train_generator,
        epochs=10,
        validation_data=valid_generator,
        callbacks=[checkpoint_callback, early_stopping_callback, tensorboard_callback]
    )

    return base_model

def extract_features(model, generator):
    features = model.predict(generator)
    labels = generator.labels
    return features, labels

if __name__ == '__main__':
    train_data_dir = "/workspace/data/data_dir/model-data/train"
    test_data_dir = "/workspace/data/data_dir/model-data/test"
    valid_data_dir = "/workspace/data/data_dir/model-data/val"

    train_generator, test_generator, valid_generator = preprocess(train_data_dir, test_data_dir, valid_data_dir)
    
    num_classes = train_generator.num_classes
    
    cnn_trained_model = train_cnn_model(train_generator, valid_generator, num_classes)
    
    # Extract features
    train_features, train_labels = extract_features(cnn_trained_model, train_generator)
    test_features, test_labels = extract_features(cnn_trained_model, test_generator)
    
    # Apply XGBoost
    xgb_classifier = xgb.XGBClassifier()
    xgb_classifier.fit(train_features, train_labels)
    accuracy = xgb_classifier.score(test_features, test_labels)
    print("XGBoost Accuracy:", accuracy)


Found 1051 images belonging to 3 classes.
Found 200 images belonging to 3 classes.
Found 99 images belonging to 3 classes.
Epoch 1/10
33/33 [==============================] - ETA: 0s - loss: 2.1237 - accuracy: 0.8230
Epoch 1: val_loss improved from inf to 199.40741, saving model to /workspace/data/data_dir/saved/best/best_model.h5
33/33 [==============================] - 20s 566ms/step - loss: 2.1237 - accuracy: 0.8230 - val_loss: 199.4074 - val_accuracy: 0.8700
Epoch 2/10
33/33 [==============================] - ETA: 0s - loss: 0.7128 - accuracy: 0.8411
Epoch 2: val_loss improved from 199.40741 to 53.05507, saving model to /workspace/data/data_dir/saved/best/best_model.h5
33/33 [==============================] - 17s 508ms/step - loss: 0.7128 - accuracy: 0.8411 - val_loss: 53.0551 - val_accuracy: 0.8700
Epoch 3/10
33/33 [==============================] - ETA: 0s - loss: 0.4616 - accuracy: 0.8421
Epoch 3: val_loss improved from 53.05507 to 3.91335, saving model to /workspace/data/data_d

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [1 2]